# Object oriented programming

Creating classes to simulate a server that's taking connections from the outside and then a load balancer that ensures that there are enough servers to serve those connections. 
<br><br>
To represent the servers that are taking care of the connections, we'll use a Server class. Each connection is represented by an id, that could, be the IP address of the computer connecting to the server.  For our simulation, each connection creates a random amount of load in the server, between 1 and 10.
<br><br>

In [2]:
import random

class Server:
    def __init__(self):
        """Creates a new server instance, with no active connections."""
        self.connections = {}

    def add_connection(self, connection_id):
        """Adds a new connection to this server."""
        connection_load = random.random()*10+1
        self.connections[connection_id] = connection_load

    def close_connection(self, connection_id):
        """Closes a connection on this server."""
        if connection_id in self.connections:
            del self.connections[connection_id]

    def load(self):
        """Calculates the current load for all connections."""
        total = 0
        for load in self.connections.values():
            total += load
        return total

    def __str__(self):
        """Returns a string with the current load of the server"""
        return "{:.2f}%".format(self.load())
    

Creating a Server instance and adding a connection to it, then checking the load:

In [3]:
server = Server()
server.add_connection("192.168.1.1")

print(server.load())

1.5595358598324118


Closing a connection with the `close_connection` method. 

In [4]:
server.close_connection("192.168.1.1")
print(server.load())


0


Alright, Let's look at the basic `LoadBalancing` class. This class will start with only one server available. When a connection gets added, it will randomly select a server to serve that connection, and then pass on the connection to the server. The `LoadBalancing` class also needs to keep track of the ongoing connections to be able to close them. This is the basic structure:

In [6]:
class LoadBalancing:
    def __init__(self):
        """Initialize the load balancing system with one server"""
        self.connections = {}
        self.servers = [Server()]

    def add_connection(self, connection_id):
        """Randomly selects a server and adds a connection to it."""
        server = random.choice(self.servers)
        server.add_connection(connection_id)
        self.ensure_availability()

    def close_connection(self, connection_id):
        """Closes the connection on the the server corresponding to connection_id."""
        for server in self.servers:
            if connection_id in server.connections:
                server.close_connection(connection_id)
                break

    def avg_load(self):
        """Calculates the average load of all servers"""
        total_load = 0
        total_server = 0
        for server in self.servers:
            total_load += server.load()
            total_server += 1
        return total_load/total_server

    def ensure_availability(self):
        """If the average load is higher than 50, spin up a new server"""
        if self.avg_load() > 50:
            self.servers.append(Server())

    def __str__(self):
        """Returns a string with the load for each server."""
        loads = [str(server) for server in self.servers]
        return "[{}]".format(",".join(loads))

Creating a connection in the load balancer, assigning it to a running server and then the load should be more than zero:

In [7]:
l = LoadBalancing()
l.add_connection("fdca:83d2::f20d")
print(l.avg_load())


5.202911081525526


Let's add a new server.

In [8]:
l.servers.append(Server())
print(l.avg_load())


2.601455540762763


The average load now is half of what it was before.
<br><br>


Closing the connection.

In [9]:
l.close_connection("fdca:83d2::f20d")
print(l.avg_load())


0.0


In [10]:
for connection in range(20):
    l.add_connection(connection)
print(l)


[74.54%,51.74%,2.70%]


The code above adds 20 new connections and then prints the loads for each server in the load balancer. 

In [11]:
print(l.avg_load())




42.99166353775508


Average load is indeed less than 50%.